# Seq2Seq 모델 만들기
### Teacher Forcing 기법을 이용합니다.
### ?를 채우기

problem으로는 seq2seq 모델만드는 코드만 짜보고, 나머지 내용 전반에 대해서는 answer를 참고하시면 됩니다.


In [ ]:
# 1. 디코딩 시작토큰 변수에 담기
? = tokenizer.token_to_id("?")

In [ ]:

class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder.to(device)
        self.decoder = decoder.to(device)
        self.device = device

    def forward(self, inputs, outputs, teacher_forcing_rate=0.99):
        """
        parameter
            inputs: 질문 - (batch, seq_length)
            outputs: 답변(정답) - (batch, seq_length)
            teacher_forcing_rate: teacher_forcing 적용 확률.
        """
        #################################
        # 1번. 차원 맞추기
        #################################

		# 1. input/output의 tensor가 1차원인 경우 해줘야할 조치
        if inputs.dim() == ?: 
            inputs = ?
        if outputs.dim() == ?:
            outputs = ?


        #################################
        # 기본 변수 설정
        #################################

        # 1.출력 문장의 길이와 배치 크기 추출
        batch_size, output_length = outputs.shape
		# 2.디코더가 생성할 수 있는 전체 단어 개수
        output_vocab_size = self.decoder.vocab_size  # 어휘사전 토큰 총 개수.
        # 3. 출력 결과(모든 시점의 예측 결과)를 저장할 텐서
        predicted_outputs = torch.zeros(output_length, batch_size, output_vocab_size).to(self.device)


        #################################
        # 2번. 인코더 실행
        #################################

        # 1.encoder를 이용해서 질문 문장의 context vector(출력 및 hidden state) 추출.
        ?, encoder_hidden = self.encoder(inputs)
        # 2.Decoder의 첫번째 hidden state로써 인코더의 마지막 출력인 Context Vector 사용
		## hint: time dimension
        decoder_hidden = ?[?].?
        # 3.Decoder에 넣을 첫번째 time step의 값: [SOS] -> [batch_size] 형태
        decoder_input = torch.full((batch_size, ), fill_value=?, device=self.device)


        #################################
        # 디코더 반복 생성
        #################################

        # 순회(반복) 하면서 단어들을 하나씩 생성.
        for t in range(?): # max_length 만큼 생성.

			# 1.디코더에 현재 입력 토큰과 hidden state를 넣어 출력 단어 확률 분포를 받음
            decoder_out, decoder_hidden = self.decoder(?, ?)

			# 2.예측된 단어의 확률 분포를 t번째 시점에 저장
            predicted_outputs[?] = decoder_out # t번째 예측 단어를 저장.


            #################################
            # Teacher Forcing 적용 여부 결정
            #################################

            # 1.teacher_forcing_rate 확률에 따라 정답을 줄지 예측을 줄지 결정(True면 정답)
            teacher_forcing = teacher_forcing_rate > random.random() # TeacherForcing 적용여부(bool)

			# 2.반복할 수록 teacher forcing 비율 감소(모델 자율성 증가)
            teacher_forcing_rate = teacher_forcing_rate * 0.99

            #################################
            # 다음 디코더 입력 준비
            #################################

            # 1.모델이 추론한 단어(예측된 단어)중 가장 확률이 높은(TOP-1) 단어의 인덱스를 추출
            top1 = decoder_out.?

            # 2.teacher forcing이면 정답 사용, 아니면 top1 예측 사용
            decoder_input = outputs[:, ?] if teacher_forcing else top1

        #################################
        # 마무리: 출력 포맷 변경
        #################################
        return predicted_outputs.transpose(1, 0)
		    #>>>> (seq_len, batch, vocab_size) -> (batch, seq_length, vocab_size) 변환.